In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from bs4 import BeautifulSoup
import ast
from utils._funcs import creating_club_dataframe,Transfmkt

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


##### Club name and id

##### Funcs

In [2]:
lst1 = ['05/06', '21/22', '10/11', '14/15']
lst_sep = []
# xx/yy - 21/22

# for lst in lst1:
#     lst_sep.append([lst])


def get_all_seasons(temp,temp_limit):
    all = []
    all.append(temp)
    if temp == temp_limit:
        pass
    else:
        seasons_n = int(temp_limit[3:]) - int(temp[:2])
        for season in range(1,seasons_n):
            if float(temp[3:]) < 9.0:
                temp = str(temp[3:]) + str('/0'+str(int(float(temp[3:])+1)))
                all.append(temp)
            else:
                temp = str(temp[3:]) + str('/'+str(int(float(temp[3:])+1)))
                all.append(temp)
    return all


get_all_seasons('00/01','21/22')

for l in lst1:
    lst_sep.append(get_all_seasons(l,'21/22'))


def get_saison(list):
    saison = []
    for item in list:
        saison.append(str('20'+item[:2]))

    return saison

def get_saison_perf_links(saisons_list,perf_links):
    all_perf_season_links = []
    for pflink in perf_links:
        perf_season_links = []
        for saison in saisons_list:
            perf_season_links.append(str(pflink + '/plus/0?saison='+saison+'&verein=&liga=&wettbewerb=&pos=&trainer_id='))
        all_perf_season_links.append(perf_season_links)
    
    return all_perf_season_links


saisons = []
for list in lst_sep:
    saisons.append(get_saison(list))

lst_final = []
for list in saisons:
    lst_final.append(get_saison_perf_links(list,lst_test))

lst_final = str(lst_final)[1:-1]
lst_final = eval(lst_final)[0]

# lst_final = get_saison_perf_links((list for list in saisons),lst_test)
lst_final

NameError: name 'lst_test' is not defined

## Transfermkt Class

In [2]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
transf = Transfmkt('se-palmeiras','1023', headers)
transf.creating_lists()

In [3]:
transf.get_players_seasons()

['Weverton',
 'Marcelo Lomba',
 'Vinicius Silvestre',
 'Gustavo Gómez',
 'Murilo',
 'Luan',
 'Henri',
 'Benjamín Kuscevic',
 'Joaquín Piquerez',
 'Jorge',
 'Vanderlan',
 'Marcos Rocha',
 'Mayke',
 'Gustavo Garcia',
 'Danilo',
 'Jailson',
 'Fabinho',
 'Gabriel Menino',
 'Zé Rafael',
 'Eduard Atuesta',
 'Raphael Veiga',
 'Gustavo Scarpa',
 'Dudu',
 'Wesley',
 'Giovani',
 'Tabata',
 'Breno Lopes',
 'Endrick',
 'Rony',
 'José Manuel López',
 'Rafael Navarro',
 'Miguel Merentiel']

In [4]:
transf.get_values_links()[3]

['/club-libertad-asuncion/kader/verein/9875/saison_id/2011',
 '/club-libertad-asuncion/kader/verein/9875/saison_id/2012',
 '/club-libertad-asuncion/kader/verein/9875/saison_id/2013',
 '/club-atletico-lanus/kader/verein/333/saison_id/2014',
 '/club-atletico-lanus/kader/verein/333/saison_id/2015',
 '/ac-milan/kader/verein/5/saison_id/2016',
 '/ac-milan/kader/verein/5/saison_id/2017',
 '/se-palmeiras/kader/verein/1023/saison_id/2018',
 '/ac-milan/kader/verein/5/saison_id/2019',
 '/se-palmeiras/kader/verein/1023/saison_id/2020',
 '/se-palmeiras/kader/verein/1023/saison_id/2021']

In [5]:
# MUDANÇAS: 1. kader pra startsite/ 2. attr
transf.get_players_values()

[[0,
  0,
  0,
  50000,
  300000,
  0,
  0,
  500000,
  500000,
  3500000,
  3000000,
  4000000,
  3000000,
  3000000,
  3500000,
  5000000,
  4500000],
 [500000,
  1000000,
  1000000,
  1000000,
  2000000,
  3200000,
  2500000,
  1500000,
  2000000,
  1400000,
  1300000,
  750000],
 [100000, 100000, 100000, 100000, 100000, 100000, 150000, 400000],
 [0, 0, 0, 500000, 1700000, 5000000, 4000000, 3500000, 0, 6000000, 8000000],
 [0, 1200000, 1000000, 2400000, 2500000, 2000000],
 [500000,
  2000000,
  3500000,
  3700000,
  2000000,
  2000000,
  1600000,
  3000000,
  3500000],
 [0, 0, 0],
 [0, 350000, 350000, 700000, 925000, 1000000, 1300000, 1500000],
 [75000, 150000, 300000, 300000, 1000000, 3000000],
 [250000, 2500000, 6000000, 12000000, 3000000, 3200000, 0, 2200000],
 [0],
 [0,
  300000,
  400000,
  250000,
  500000,
  3000000,
  3000000,
  5000000,
  6000000,
  5000000,
  3500000,
  2500000,
  2000000,
  1500000,
  1500000],
 [150000,
  2500000,
  5000000,
  2000000,
  1000000,
  120000

In [7]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# endereco_da_pagina stands for the data page address
endereco_da_pagina = "https://www.transfermarkt.com.br/sc-corinthians/kader/verein/199/saison_id/2005"

# In the objeto_response variable we will the download of the web page
objeto_response = requests.get(endereco_da_pagina, headers=headers)

"""
Now we will create a BeautifulSoup object from our object_response.
The 'html.parser' parameter represents which parser we will use when creating our object,
a parser is a software responsible for converting an entry to a data structure.
"""
pagina_bs = BeautifulSoup(objeto_response.content, 'html.parser')

In [13]:
transfmkt_site = 'https://www.transfermarkt.com.br'

# Players names
# nomes_jog = [] # List that will receive all the players names - check
# tags_nomes = pagina_bs.find_all("span", {"class": "show-for-small"})
# for tag_nome in tags_nomes:
#     nomes_jog.append(tag_nome.text)

tags_even = pagina_bs.find_all("tr", {"class": "even"})
tags_odd = pagina_bs.find_all("tr", {"class": "odd"})
# tags_posrela = pagina_bs.find_all("td", {"class": "posrela"})
all_tags = tags_even + tags_odd
all_tags_att = BeautifulSoup(str(all_tags).strip('[]'), 'html.parser')
#name = self.nomes_jog[cont_names]
lista = []
for tag in all_tags_att.find_all("td", {"class": "posrela"}):#[0]#.img['alt']
    #lista.append(tag.text)
    lista.append(str(tag.a.text).strip())

try:
    name_turn = lista.index('Gustavo Nery')
    player_tag = BeautifulSoup(str(all_tags[name_turn]).strip('[]'), 'html.parser')
    player_value = player_tag.find("td", {"class": "rechts hauptlink"}).text
    #player_value = '1,40 mi. €'
    if 'mil €' in player_value:
        player_value = int(str(player_value.split()[0] + '000'))
    else:
        num_bef = str(player_value.split()[0].split(',')[0])
        num_after = str(player_value.split()[0].split(',')[1][0])
        player_value = int(num_bef + num_after + '00000')
except:
    player_value = 0

    #player_value = int(str(player_value.split()[0].split(',')[0] + '000'))
# for name in lista:
#     if 'Weverton' in name:
#         print(True)
#     else:
#         print(False)
print(player_value)
lista

2500000


['Marcelo',
 'Betão',
 'Sebastián Domínguez',
 'Marcus Vinícius',
 'Gustavo Nery',
 'Eduardo Ratinho',
 'Fagner',
 'Magrão',
 'Xavier',
 'Rafael Fefo',
 'Wendel',
 'Rosinei',
 'Carlos Alberto',
 'Marcelinho Carioca',
 'Ramón',
 'Daniel Grando',
 'Nilmar',
 'Nadson',
 'Rafael Moura',
 'Igor Siqueira',
 'Johnny Herrera',
 'Silvio Luiz',
 'Marinho',
 'Wescley',
 'Marquinhos',
 'Rubens Júnior',
 'Coelho',
 'Javier Mascherano',
 'Paulo Almeida',
 'Marcelo Mattos',
 'Edson Sitta',
 'Bruno Octávio',
 'César',
 'Ricardinho',
 'Roger',
 'Willian',
 'Carlos Tevez',
 'Márcio Amoroso',
 'Wilson',
 'Renato Ribeiro']

In [8]:
tags_transf_clubs_all = pagina_bs.find_all("div", {"class": "tm-player-transfer-history-grid__new-club"})
tags_transf_clubs = BeautifulSoup(str(tags_transf_clubs_all).strip('[]'), 'html.parser')

# SOLUÇÃO!!!!!
transf_clubs = []
for a in tags_transf_clubs.find_all('a', href = True):
    if 'sem-clube' in a.get('href'):
        transf_clubs.append(a.get('href'))
        transf_clubs.append('x')
    else:
        transf_clubs.append(a.get('href'))

pares = []
for c in range(0,len(transf_clubs)):
    if c % 2 == 0:
        pares.append(c)
    else:
        pass

transf_clubs_att = []
for par in pares:
    transf_clubs_att.append(transf_clubs[par])
    
transf_clubs_att

['/se-palmeiras/transfers/verein/1023/saison_id/2019',
 '/athletico-paranaense/transfers/verein/679/saison_id/2018',
 '/sem-clube/transfers/verein/515/saison_id/2017',
 '/albirex-niigata/transfers/verein/8344/saison_id/2016',
 '/cruzeiro-ec/transfers/verein/609/saison_id/2016',
 '/nautico/transfers/verein/2646/saison_id/2015',
 '/cruzeiro-ec/transfers/verein/609/saison_id/2014']